Задание 1.

Обучите нейронную сеть решать шифр Цезаря.

Что необходимо сделать:

* Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
* Сделать нейронную сеть
* Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)
* Проверить качество

Задание 2.

Выполнить практическую работу из лекционного ноутбука.

* Построить RNN-ячейку на основе полносвязных слоев
* Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

In [101]:
import pandas as pd  # для работы с данными
import time  # для оценки времени
import torch  # для написания нейросети
import numpy as np
import re

In [102]:
abvgd = 'abcdefghijklmnopqrstuvwxyz '
len(abvgd)

27

In [129]:
def f_cifer_cesar(phrase,num_K):

    abvgd = 'abcdefghijklmnopqrstuvwxyz'
    if num_K>len(abvgd):
        num_K=len(abvgd)

    ab_ba=abvgd+abvgd[::-1]

    w_i_dict = {w: i for i, w in enumerate(abvgd)}

    loc_lst=''
    for letter in phrase:
        if letter in abvgd:
            loc_lst+=ab_ba[w_i_dict[letter]+num_K]
        else:
            loc_lst+=letter

    return loc_lst

In [130]:
def f_clean_text(phrase):

    abvgd = 'abcdefghijklmnopqrstuvwxyz '

    w_i_dict = {w: i for i, w in enumerate(abvgd)}

    loc_lst=''
    for letter in phrase.lower():
        if letter in abvgd:
            loc_lst+=abvgd[w_i_dict[letter]]
        else:
            loc_lst+=''

    return loc_lst

In [131]:
def f_text_to_num(phrase):

    abvgd = 'abcdefghijklmnopqrstuvwxyz '

    w_i_dict = {w: i for i, w in enumerate(abvgd)}

    loc_lst=[]
    for letter in phrase.lower():
        if letter in abvgd:
            loc_lst.append(w_i_dict[letter])

    return torch.Tensor(loc_lst)

In [132]:
def f_num_to_text(lst):

    abvgd = 'abcdefghijklmnopqrstuvwxyz '
    lst=list(lst.detach().numpy())
    i_w_dict = {i: w for i, w in enumerate(abvgd)}

    loc_lst=''
    for num in lst:
        if num in range(len(abvgd)):
            loc_lst+=i_w_dict[num]

    return loc_lst

In [133]:
def f_cadr_text(phrase,lenght):

    loc_lst=phrase[:50]
    #for i in range(50):#range(lenght-len(phrase)):
        #loc_lst+=' '

    return loc_lst

In [134]:
df = pd.read_csv('data.csv',usecols=['normalized_text'])

df=df.dropna().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10891 entries, 0 to 10890
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   normalized_text  10891 non-null  object
dtypes: object(1)
memory usage: 85.2+ KB


In [135]:
df['clean_text']=[f_clean_text(x) for x in df['normalized_text']]
df['len_text']=[len(x) for x in df['clean_text']]
df=df[df['len_text']>50].reset_index(drop=True)
df.head()


,normalized_text,clean_text,len_text
0,im trying to teach maggie that nature doesnt e...,im trying to teach maggie that nature doesnt e...,122
1,its like an ox only it has a hump and a dewlap...,its like an ox only it has a hump and a dewlap...,78
2,ooh look maggie what is that do-dec-ah-edron d...,ooh look maggie what is that dodecahedron dode...,54
3,thats okay bart nobody really believed it we w...,thats okay bart nobody really believed it we w...,74
4,bart my birthday is in two days im gonna be ei...,bart my birthday is in two days im gonna be ei...,98


In [136]:
df.info

<bound method DataFrame.info of                                         normalized_text  \
0     im trying to teach maggie that nature doesnt e...   
1     its like an ox only it has a hump and a dewlap...   
2     ooh look maggie what is that do-dec-ah-edron d...   
3     thats okay bart nobody really believed it we w...   
4     bart my birthday is in two days im gonna be ei...   
...                                                 ...   
3799  i think its sad that you equate friendliness f...   
3800  perhaps he realized how hollow the pursuit of ...   
3801  too bad we didnt come dressed as popular carto...   
3802  yeah mom guess what for a dollar a man sold me...   
3803  maybe you need to play on their sympathies mor...   

                                             clean_text  len_text  
0     im trying to teach maggie that nature doesnt e...       122  
1     its like an ox only it has a hump and a dewlap...        78  
2     ooh look maggie what is that dodecahedron dode...

In [137]:
lenght_max=50

In [138]:
len(f_cadr_text(df['clean_text'][0],lenght_max))

50

In [139]:
df['cadr_text']=[f_cadr_text(x,lenght_max) for x in df['clean_text']]

In [140]:
K_cifer=25

In [141]:
df['cifer_text']=[f_cifer_cesar(x,K_cifer) for x in df['cadr_text']]
df.head()

,normalized_text,clean_text,len_text,cadr_text,cifer_text
0,im trying to teach maggie that nature doesnt e...,im trying to teach maggie that nature doesnt e...,122,im trying to teach maggie that nature doesnt e...,so hjcsnu hm hwzyt ozuusw htzh nzhgjw xmwinh w...
1,its like an ox only it has a hump and a dewlap...,its like an ox only it has a hump and a dewlap...,78,its like an ox only it has a hump and a dewlap...,shi psqw zn md mnpc sh tzi z tgol znx z xwepzl...
2,ooh look maggie what is that do-dec-ah-edron d...,ooh look maggie what is that dodecahedron dode...,54,ooh look maggie what is that dodecahedron dode...,mmt pmmq ozuusw etzh si htzh xmxwyztwxjmn xmxw...
3,thats okay bart nobody really believed it we w...,thats okay bart nobody really believed it we w...,74,thats okay bart nobody really believed it we w...,htzhi mqzc zzjh nmzmxc jwzppc zwpswfwx sh ew e...
4,bart my birthday is in two days im gonna be ei...,bart my birthday is in two days im gonna be ei...,98,bart my birthday is in two days im gonna be ei...,zzjh oc zsjhtxzc si sn hem xzci so umnnz zw ws...


In [142]:
df=df[['cadr_text','cifer_text']]
df.head()

,cadr_text,cifer_text
0,im trying to teach maggie that nature doesnt e...,so hjcsnu hm hwzyt ozuusw htzh nzhgjw xmwinh w...
1,its like an ox only it has a hump and a dewlap...,shi psqw zn md mnpc sh tzi z tgol znx z xwepzl...
2,ooh look maggie what is that dodecahedron dode...,mmt pmmq ozuusw etzh si htzh xmxwyztwxjmn xmxw...
3,thats okay bart nobody really believed it we w...,htzhi mqzc zzjh nmzmxc jwzppc zwpswfwx sh ew e...
4,bart my birthday is in two days im gonna be ei...,zzjh oc zsjhtxzc si sn hem xzci so umnnz zw ws...


In [143]:
len(df['cadr_text'][0])

50

In [144]:
df['cifer_vec']=[f_text_to_num(x) for x in df['cifer_text']]
df['cadr_vec']=[f_text_to_num(x) for x in df['cadr_text']]



<ipython-input-144-9731b690fcf2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cifer_vec']=[f_text_to_num(x) for x in df['cifer_text']]
<ipython-input-144-9731b690fcf2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cadr_vec']=[f_text_to_num(x) for x in df['cadr_text']]


In [145]:
(df['cifer_vec'][0].shape)
(df['cifer_vec'][0])

tensor([18., 14., 26.,  7.,  9.,  2., 18., 13., 20., 26.,  7., 12., 26.,  7.,
        22., 25., 24., 19., 26., 14., 25., 20., 20., 18., 22., 26.,  7., 19.,
        25.,  7., 26., 13., 25.,  7.,  6.,  9., 22., 26., 23., 12., 22.,  8.,
        13.,  7., 26., 22., 13., 23., 26.,  4.])

In [146]:
df.head()

,cadr_text,cifer_text,cifer_vec,cadr_vec
0,im trying to teach maggie that nature doesnt e...,so hjcsnu hm hwzyt ozuusw htzh nzhgjw xmwinh w...,"[tensor(18.), tensor(14.), tensor(26.), tensor...","[tensor(8.), tensor(12.), tensor(26.), tensor(..."
1,its like an ox only it has a hump and a dewlap...,shi psqw zn md mnpc sh tzi z tgol znx z xwepzl...,"[tensor(18.), tensor(7.), tensor(8.), tensor(2...","[tensor(8.), tensor(19.), tensor(18.), tensor(..."
2,ooh look maggie what is that dodecahedron dode...,mmt pmmq ozuusw etzh si htzh xmxwyztwxjmn xmxw...,"[tensor(12.), tensor(12.), tensor(19.), tensor...","[tensor(14.), tensor(14.), tensor(7.), tensor(..."
3,thats okay bart nobody really believed it we w...,htzhi mqzc zzjh nmzmxc jwzppc zwpswfwx sh ew e...,"[tensor(7.), tensor(19.), tensor(25.), tensor(...","[tensor(19.), tensor(7.), tensor(0.), tensor(1..."
4,bart my birthday is in two days im gonna be ei...,zzjh oc zsjhtxzc si sn hem xzci so umnnz zw ws...,"[tensor(25.), tensor(25.), tensor(9.), tensor(...","[tensor(1.), tensor(0.), tensor(17.), tensor(1..."


In [165]:
from sklearn.model_selection import  train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['cifer_text'], df['cadr_text'], test_size=0.2, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(df['cifer_vec'], df['cadr_vec'], test_size=0.2, random_state=42)

In [166]:
X_train=X_train.reset_index(drop=True)
X_test=X_test.reset_index(drop=True)
y_train=y_train.reset_index(drop=True)
y_test=y_test.reset_index(drop=True)

In [167]:
X_train[0]

'ewpp so zvjzsx htw pze si ypwzj znx xzx xsx jwnmgn'

In [124]:
'''X_train=pd.Series([torch.Tensor(x).reshape(1,1,lenght_max) for x in X_train])
X_test= pd.Series([torch.Tensor(x).reshape(1,1,lenght_max) for x in  X_test])
y_train=pd.Series([torch.Tensor(x).reshape(1,1,lenght_max) for x in y_train])
y_test= pd.Series([torch.Tensor(x).reshape(1,1,lenght_max) for x in  y_test])'''

'X_train=pd.Series([torch.Tensor(x).reshape(1,1,lenght_max) for x in X_train])\nX_test= pd.Series([torch.Tensor(x).reshape(1,1,lenght_max) for x in  X_test])\ny_train=pd.Series([torch.Tensor(x).reshape(1,1,lenght_max) for x in y_train])\ny_test= pd.Series([torch.Tensor(x).reshape(1,1,lenght_max) for x in  y_test])'

In [168]:
type(X_train[0])

str

In [169]:
len(y_train),len(X_train),len(y_test),len(X_test)

(3043, 3043, 761, 761)

In [171]:
window_size=50
def input_data(seq_x,seq_y,ws=window_size):
    out = []
    L = len(seq_x)
    
    for i in range(L):
        window = seq_x[i]
        label = seq_y[i]
        out.append((window,label))
    
    return out

In [172]:

train_data = input_data(X_train,y_train,ws=window_size)
test_data = input_data(X_test,y_test,ws=window_size)


In [173]:
type(train_data[0])

tuple

In [175]:
train_data[0][1]

'well im afraid the law is clear and dad did renoun'

In [176]:
import torch.optim.lr_scheduler as lr_scheduler

In [254]:
class LSTM(torch.nn.Module):
    
    def __init__(self,input_size = 27, hidden_size = 256, out_size = 27):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = torch.nn.LSTM(input_size, hidden_size)
        self.linear = torch.nn.Linear(hidden_size,out_size)
        self.hidden = (torch.zeros(1,1,hidden_size),torch.zeros(1,1,hidden_size))
    
    def forward(self,seq):
        #lstm_out, self.hidden = self.lstm(seq.view(len(seq),1,-1), self.hidden)
        lstm_out, self.hidden = self.lstm(seq.view(1,1,27), self.hidden)
        pred = self.linear(lstm_out.view(256))
        return pred#[-1]

In [255]:
torch.manual_seed(42)
model = LSTM()
criterion = torch.nn.MSELoss()#reduction='sum'
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler1 = lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=0.00001)

In [179]:
model

LSTM(
  (lstm): LSTM(27, 256)
  (linear): Linear(in_features=256, out_features=27, bias=True)
)

In [212]:
for seq, y_train in train_data:
    print((seq),'\n',(y_train))
    break

ewpp so zvjzsx htw pze si ypwzj znx xzx xsx jwnmgn 
 well im afraid the law is clear and dad did renoun


In [183]:
torch.eye(3,3)[2]

tensor([0., 0., 1.])

In [184]:
train_data[:1][:]

[('ewpp so zvjzsx htw pze si ypwzj znx xzx xsx jwnmgn',
  'well im afraid the law is clear and dad did renoun')]

In [213]:
abvgd = 'abcdefghijklmnopqrstuvwxyz '

w_i_dict = {w: i for i, w in enumerate(abvgd)}

vectors_letter=torch.eye(27,27)

for x_seq, y_seq in train_data[:1][:]:
    for x_seq_letter, y_seq_letter in zip(x_seq, y_seq):
            
            x_seq_letter=vectors_letter[w_i_dict[x_seq_letter]]
            y_seq_letter=vectors_letter[w_i_dict[y_seq_letter]]
            print(x_seq_letter.shape,'\n',y_seq_letter.shape,'\n')
            print(len(x_seq_letter),'\n',y_seq_letter,'\n')
            break

torch.Size([27]) 
 torch.Size([27]) 

27 
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0.]) 



In [ ]:
epochs = 5

for i in range(epochs):
    
    for x_seq, y_seq in train_data[:30][:]:
        for x_seq_letter, y_seq_letter in zip(x_seq, y_seq):
            
            x_seq_letter=vectors_letter[w_i_dict[x_seq_letter]]
            y_seq_letter=vectors_letter[w_i_dict[y_seq_letter]]
            #print(x_seq_letter,'\n',y_seq_letter,'\n')

            optimizer.zero_grad()
            model.hidden = (torch.zeros(1,1,model.hidden_size),
                        torch.zeros(1,1,model.hidden_size))
            
            y_pred = model(x_seq_letter)
            loss = criterion(y_pred.squeeze(), y_seq_letter.squeeze())
            loss.backward()
            optimizer.step()
        
    
    scheduler1.step()
    
    for x_seq, y_seq in test_data[:10][:]:  
        for x_seq_letter, y_seq_letter in zip(x_seq, y_seq):  

            x_seq_letter=vectors_letter[w_i_dict[x_seq_letter]]
            y_seq_letter=vectors_letter[w_i_dict[y_seq_letter]]

            y_pred = model(x_seq_letter.squeeze())
            
            loss_test = criterion(y_pred.squeeze(), y_seq_letter.squeeze())
            
    if (i+1) % 1 == 0:
        print(f"Epoch {i+1} MSE_TRAIN_Loss: {loss.item()}")
        print(f"          MSE_TEST__Loss: {loss_test}")

In [269]:
i=0
Accuracy=0
for x_seq, y_seq in test_data[:5][:]:  
        
        i+=1
        for x_seq_letter, y_seq_letter in zip(x_seq, y_seq):  

            x_seq_letter=vectors_letter[w_i_dict[x_seq_letter]]
            y_seq_letter=vectors_letter[w_i_dict[y_seq_letter]]

            y_pred = model(x_seq_letter)
            
            loss_test = criterion(y_pred.squeeze(), y_seq_letter.squeeze())
            if torch.argmax(y_seq_letter)==torch.argmax(y_pred):
                Accuracy+=1

        print('Accuracy',Accuracy/(50*i))

Accuracy 1.0
Accuracy 0.99
Accuracy 0.98
Accuracy 0.985
Accuracy 0.988


In [304]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [ ]:
phrases = df['normalized_text'].tolist()  # колонка с предобработанными текстами
phrases[:3]

['maggie look whats that', 'lee-mur lee-mur', 'zee-boo zee-boo']

In [ ]:
text = [[c for c in ph] for ph in phrases[:3] if type(ph) is str]
text

In [ ]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')  # все символы, которые мы хотим использовать для кодировки = наш словарь
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]  # все неизвестные символы будут получать тег none
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}  # словарь токен-индекс

In [ ]:
INDEX_TO_CHAR

In [ ]:
MAX_LEN = 50  # мы хотим ограничить максимальную длину ввода
X = torch.zeros((len(text), MAX_LEN), dtype=int)  # создаём пустой вектор для текста, чтобы класть в него индексы токенов
for i in range(len(text)):  # для каждого предложения
    for j, w in enumerate(text[i]):  # для каждого токена
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])